In [ ]:
import scipy.integrate as integrate
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
import pandas as pd
from pandas import DataFrame, Series

# Solving the system of differential equations
$x$ = stretch = Spring stretch (meters)<br>
$\theta$ = angle = Angle from vertical (radians)<br>
$m$ = mass = Pendulum's mass (kilograms)<br>
$k$ = spring = Spring constant (Newtons/meter)<br>
$l$ = length = Unstretched pendulum length (meters)<br>
$g$ = gravity = Acceleration due to gravity (meters/second^2)<br>
<br>
System of 2nd order equations given: <br>
$\ \ \ \ \ \ x'' + \frac{k}{m}x - (l+x)\theta'^2 - g \cos\theta = 0$ <br>
$\ \ \ \ \ \ \theta'' + \frac{g \sin\theta + 2x'\theta'}{l+x} = 0$ <br>
<br>
Converted to system of 1st order equations: <br>
$ \qquad y_{1} = x \qquad \qquad y_{1} ' = y_{2} = x' $ <br>
$ \qquad y_{2} = x' \qquad \qquad y_{2}' = x'' = -\frac{k}{m}y_{1} + (l+y_{1})(y_{4})^2 + g \cos y_{3} $ <br>
$ \qquad y_{3} = \theta \qquad \qquad y_{3}' = y_{4} = \theta' $ <br>
$ \qquad y_{4} = \theta' \qquad \qquad y_{4}' = \theta'' = -\frac{g \sin y_{3} - 2y_{2}y_{4}}{l+y_{1}} $

In [ ]:
# Function used for solving the system of differential equations
def pend(I, t, m, k, l, g): 
    dy1dt = I[1]                                                  # y1' = y2 = x'
    dy2dt = (-k/m)*I[0] + (l+I[0])*(I[3])**2 + g*math.cos(I[2])   # y2' = x"
    dy3dt = I[3]                                                  # y3' = y4 = theta' 
    dy4dt = (-g*math.sin(I[2]) - (2.0*I[1]*I[3])) / (l+I[0])      # y4' = theta" 
    return [dy1dt, dy2dt, dy3dt, dy4dt]

# Performing Monte Carlo simulation

Define arguments in the next cell.<br>
Each run will yield a separate file, each with the designated number of trials.<br>
Files are saved as ParameterDataNUM.csv, where NUM begins with the fileNum designated below.<br>
Row and column headers are included in the csv, each row is a trial, and each column is a parameter, with the classification column last.

In [ ]:
totTime = 30 # Total time for each trial for solving the differential equations (in seconds)
numSteps = 3000 # Number of time steps to take in totTime (step size=totTime/numSteps)
numTrials = 200 # Number of trials for each run
numRuns = 5 # Number of runs to perform (total trials performed = numRuns * numTrials)
fileNum = 0 # For beginning the file numbering

In [ ]:
t = np.linspace(0, totTime, numSteps) # Time array

# Run the Monte Carlo
for run in range(numRuns):
    
    # Initialize data frames
    parameterData = DataFrame() # Stores the parameters and classifications for each trial

    # Separated for graphing purposes:
    angleData = DataFrame() # For the angles during the successful trials
    angleFailureData = DataFrame() # For the angles during the failed trials
    stretchData = DataFrame() # For the stretches during the successful trials
    stretchFailureData = DataFrame() # For the stretches during the failed trials
    
    for trial in range(numTrials):
    
        # Define parameters
        mass = norm.rvs(1.0, 0.1)             # Pendulum's mass (kilograms)
        spring = norm.rvs(30.0, 0.25)         # Spring constant (Newtons/meter)
        length = norm.rvs(1.0, 0.1)           # Unstretched pendulum length (meters)
        gravity = norm.rvs(9.8, 0.1)          # Acceleration due to gravity (meters/second^2)
    
        # Define initial conditions
        initStretch = norm.rvs(0.1, 0.01)     # Initial stretch (meters)
        initStretchPrime = 0                  # Initial rate of change in stretch (meters/second)
        initAngle = norm.rvs(0.175, 0.01)     # Initial angle of swing from vertical (radians) 
                                              #      (about N(10, 0.6) in degrees)
        initAnglePrime = 0                    # Initial rate of change in angle (radians/second)
    
        # Combine initial conditions into a single array
        initial_conditions = [initStretch, initStretchPrime, initAngle, initAnglePrime]    

        #-------------------------------------
        # Solve the differential equations
        results = DataFrame(integrate.odeint(pend, initial_conditions, t, 
                                             args=(mass, spring, length, gravity)))    
    
        # Get the angles and stretches during the trial
        stretchSeries = Series(results[0])
        angleSeries = Series(results[2])
   
        # Define performance metric and classify results
        if (angleSeries.max() > math.radians(21)): # Failure if max angle exceeded 21 degrees     
            angleFailureData = pd.concat( [angleFailureData, angleSeries], axis=1 )
            angleFailureData = angleFailureData.rename( columns={ 2:'trial'+str(trial) } )
            stretchFailureData = pd.concat( [stretchFailureData, stretchSeries], axis=1 )
            stretchFailureData = stretchFailureData.rename( columns={ 0:'Trial '+str(trial) } )
            parameterData = pd.concat( [parameterData, Series( {'Mass':mass, 'Spring':spring, 
                                       'Length':length, 'Gravity':gravity, 
                                       'Initial Stretch':initStretch, 'Initial Angle':initAngle,
                                       'Class':0} ) ], axis=1 )
            parameterData = parameterData.rename( columns={ 0:'Trial '+str(trial) } )        
        
        else: # Success (max angle <= 21 degrees)
            angleData = pd.concat( [angleData, angleSeries], axis=1 ) 
            angleData = angleData.rename( columns={ 2:'Trial '+str(trial) } )    
            stretchData = pd.concat( [stretchData, stretchSeries], axis=1 )
            stretchData = stretchData.rename( columns={ 0:'Trial '+str(trial) } )
            parameterData = pd.concat( [parameterData, Series( {'Mass':mass, 'Spring':spring,
                                       'Length':length, 'Gravity':gravity,
                                       'Initial Stretch':initStretch, 'Initial Angle':initAngle,
                                       'Class':1} ) ], axis=1 )
            parameterData = parameterData.rename( columns={ 0:'Trial '+str(trial) } )

    #--------------------------------
    # Move 'Class' to the end in the parameter data frame and save the parameter data for this run
    cols = list(parameterData.T)
    cols.insert( len(cols)-1, cols.pop(cols.index('Class')) )
    parameterData = parameterData.T.loc[:, cols]
    parameterData.to_csv('ParameterData'+str(fileNum+run)+'.csv', encoding='utf-8')

# Plot angle over time

###### Degrees

In [ ]:
plt.figure( figsize=(15, 5) )
plt.title('Angle vs Time')
plt.xlabel('Time (s)')
plt.ylabel('Angle (degrees)')
plt.grid()

# Plot successes
(rowCount, colCount) = angleData.shape
for colNum in range(colCount):
    radianAngles = angleData.iloc[:, colNum].values
    degreeAngles = [ math.degrees(x) for x in radianAngles ]
    plt.plot(t, degreeAngles, 'b') 
  
# Plot failures
(rowCount, colCount) = angleFailureData.shape  
for colNum in range(colCount):
    radianAngles = angleFailureData.iloc[:, colNum].values
    degreeAngles = [ math.degrees(x) for x in radianAngles ]
    plt.plot(t, degreeAngles, 'r')

plt.savefig('AngleDegreesPlot.png')
plt.show()

###### Radians

In [ ]:
plt.figure( figsize=(15, 5) )
plt.title('Angle vs Time')
plt.xlabel('Time (s)')
plt.ylabel('Angle (radians)')
plt.grid()

# Plot successes
(rowCount, colCount) = angleData.shape
for colNum in range(colCount):
    plt.plot(t, angleData.iloc[:, colNum], 'b')     
  
# Plot failures
(rowCount, colCount) = angleFailureData.shape  
for colNum in range(colCount):
    plt.plot(t, angleFailureData.iloc[:, colNum], 'r')

plt.savefig('AngleRadiansPlot.png')   
plt.show()

# Plot stretch over time

In [ ]:
plt.figure( figsize=(15,5) )
plt.title('Stretch vs Time')
plt.xlabel('Time (s)')
plt.ylabel('Stretch (m)')
plt.grid()

# Plot successes
(rowCount, colCount) = stretchData.shape
for colNum in range(colCount):
    col = stretchData.iloc[:, colNum]
    plt.plot(t, col, 'b')
  
# Plot failures
(rowCount, colCount) = stretchFailureData.shape  
for colNum in range(colCount):
    col = stretchFailureData.iloc[:, colNum]
    plt.plot(t, col, 'r')

plt.savefig('StretchPlot.png')
plt.show()